SICP 习题 （2.11）解题总结:区间乘法的优化

SICP 习题 2.11又出现Ben这个人了，如以前说到的，只要是Ben说的一般都是对的。

来看看Ben说什么，他说：“通过监测区间的端点，有可能将mul-interval分解为9中情况，每种情况中所需要的乘法都不超过两次”。

所以这个叫Ben的人建议Allysa重写mul-interval过程。

到底是啥意思呢，我们先来看看以前的mul-interval过程：

In [1]:
(define (mul-interval x y)
  (let (( p1 (* (lower-bound x) (lower-bound y)))
	( p2 (* (lower-bound x) (upper-bound y)))
	( p3 (* (upper-bound x) (lower-bound y)))
	( p4 (* (upper-bound x) (upper-bound y))))
    (make-interval (min p1 p2 p3 p4)
		   (max p1 p2 p3 p4))))

可以发现，这里使用了4次乘法，然后取4此乘法的最小值为起点，最大值为终点。

按Ben的意思，我们可以将这4次乘法减少为两次，前提是对区间的端点进行判断。

其实我们自己想一想大概能够明白Ben这段神秘的话。 比如，如果相乘的两个区间都是完全大于零的区间，两个区间的起点相乘肯定是4次乘法中最小的值，而两个终点相乘肯定是4次乘法中的最大的，这样我们只需要计算两个起点相乘，还有就是两个终点相乘就可以了。这样我们就可以使用2次乘法完成工作，而不用4次。

不过，对我们程序员来讲工作就复杂很多了，我们需要取判断这9中情况，分别想好9种情况种选用什么作为结构的起点和终点，最后写出来的代码如下，巨烦琐：

In [2]:
(define (mul-interval x y)
  (if (> (lower-bound x) 0)
      (if (> (lower-bound y) 0)
	  (make-interval (* (lower-bound x) (lower-bound y)) (* (upper-bound x) (upper-bound y)))
	  (if (> (upper-bound y) 0)
	      (make-interval (* (upper-bound x) (lower-bound y)) (* (upper-bound x) (upper-bound y)))
	      (make-interval (* (lower-bound x) (upper-bound y)) (* (lower-bound x) (upper-bound y)))))
      (if (> (upper-bound x) 0)
	  (if (> (lower-bound y) 0)
	      (make-interval (* (lower-bound x) (upper-bound y)) (* (upper-bound x) (upper-bound y)))
	      (if (> (upper-bound y) 0)
		  (make-interval (* (lower-bound x) (lower-bound y)) 
				 (* (upper-bound x) (upper-bound y)))
		  (make-interval (* (lower-bound x) (lower-bound y))
				 (* (upper-bound x) (upper-bound y)))))
	  (if (> (lower-bound y) 0)
	      (make-interval (* (lower-bound x) (lower-bound y)) (* (upper-bound x) (upper-bound y)))
	      (if (> (upper-bound y) 0)
		  (make-interval (* (lower-bound x) (lower-bound y)) 
				 (* (upper-bound x) (upper-bound y)))
		  (make-interval (* (lower-bound x) (lower-bound y)) 
				 (* (upper-bound x) (upper-bound y))))) )))

有人可能会问，把原来那个如此优雅的过程写成现在这样有意思吗？一堆丑陋的判断！

这里需要理解的就是，如果系统中乘法是一个消耗很大的操作，比如每个乘法消耗2秒，这样我们做这个优化就有意义的，虽然我们写的代码丑很多，麻烦很多，不过代码运行效率就比较高了。

In [5]:
(define (make-interval a b)
  (cons a b))

(define (lower-bound x)
  (car x))

(define (upper-bound x)
  (cdr x))


(define (add-interval x y)
  (make-interval (+ (lower-bound x) (lower-bound y))
		 (+ (upper-bound x) (upper-bound y))))

(define (sub-interval x y)
  (make-interval (- (lower-bound x) (lower-bound y))
		 (- (upper-bound x) (upper-bound y))))




(define (mul-interval x y)
  (if (> (lower-bound x) 0)
      (if (> (lower-bound y) 0)
	  (make-interval (* (lower-bound x) (lower-bound y)) (* (upper-bound x) (upper-bound y)))
	  (if (> (upper-bound y) 0)
	      (make-interval (* (upper-bound x) (lower-bound y)) (* (upper-bound x) (upper-bound y)))
	      (make-interval (* (lower-bound x) (upper-bound y)) (* (lower-bound x) (upper-bound y)))))
      (if (> (upper-bound x) 0)
	  (if (> (lower-bound y) 0)
	      (make-interval (* (lower-bound x) (upper-bound y)) (* (upper-bound x) (upper-bound y)))
	      (if (> (upper-bound y) 0)
		  (make-interval (* (lower-bound x) (lower-bound y)) 
				 (* (upper-bound x) (upper-bound y)))
		  (make-interval (* (lower-bound x) (lower-bound y))
				 (* (upper-bound x) (upper-bound y)))))
	  (if (> (lower-bound y) 0)
	      (make-interval (* (lower-bound x) (lower-bound y)) (* (upper-bound x) (upper-bound y)))
	      (if (> (upper-bound y) 0)
		  (make-interval (* (lower-bound x) (lower-bound y)) 
				 (* (upper-bound x) (upper-bound y)))
		  (make-interval (* (lower-bound x) (lower-bound y)) 
				 (* (upper-bound x) (upper-bound y))))) )))


(define (div-interval x y)
  (if (< (* (upper-bound y) (lower-bound y)) 0)
      (error "Div 0:" y))
  (mul-interval x
		(make-interval (/ 1.0 (upper-bound y))
			       (/ 1.0 (lower-bound y)))))


(define (interval-width interval)
  (/ (-  (upper-bound interval) (lower-bound interval)) 2))




(define (start-test-2-11)
  (mul-interval (make-interval 1 2) (make-interval -1 3)))


In [6]:
(start-test-2-11)

(-2 . 6)